In [186]:
# Mode shares to Seattle RGCs (2006 survey)
import pandas as pd
import h5py
pd.options.mode.chained_assignment = None  # default='warn'

In [187]:
# import data (using the 2006 survey h5 input for soundcast)
h5_file = r'R:\SoundCast\Inputs\2010\etc\survey.h5'
h5 = h5py.File(h5_file)

In [188]:
def h5_to_df(h5,table):
    df = pd.DataFrame()
    for col in h5[table].keys():
        df[col] = [i[0] for i in h5[table][col][:]]
        
    return df

In [189]:
# Import h5 tables as dataframes
hh = h5_to_df(h5,'Household')
trips = h5_to_df(h5,'Trip')
print len(trip)
print len(hh)

11128
4741


In [190]:
trips.columns

Index([u'arrtm', u'dadtyp', u'day', u'deptm', u'dorp', u'dpcl', u'dpurp', u'dtaz', u'endacttm', u'half', u'hhno', u'mode', u'oadtyp', u'opcl', u'opurp', u'otaz', u'pathtype', u'pno', u'tour', u'travcost', u'travdist', u'travtime', u'trexpfac', u'tseg', u'tsvid'], dtype='object')

In [191]:
# Add lookup for RGC to trip destination parcel
# Using file stored on R drive, sorry about the hard codes :/
rgc_lookup =pd.read_csv(r'R:\SoundCast\releases\TransportationFutures2010\scripts\summarize\parcels_in_urbcens.csv')

# rgc_lookup.groupby('NAME').count()

In [192]:
# Join to trip file based on trip destination parcel
# returns only trips that end within an RGC
trips = trips.merge(rgc_lookup, left_on='dpcl', right_on='hhparcel', how='inner')
print len(trips)

# rename the NAME column to something more understandable
trips['d_rgc_name'] = trips['NAME']

11128


In [193]:
# Join RGC to household location
hh = hh.merge(rgc_lookup, left_on='hhparcel',right_on='hhparcel',how='inner')
hh['hh_rgc_name'] = hh['NAME']
print len(hh)

223


In [194]:
# Join household data to trips
trip_hh = trips.merge(hh,on='hhno')


In [195]:
seattle_rgcs = ['Seattle Downtown', 'Seattle First Hill/Capitol Hill', 'Seattle Northgate',
             'Seattle South Lake Union','Seattle University Community','Seattle Uptown']

In [196]:
def format_pivot(df):
    '''
    Format pivot table results to include Seattle RGCs, select modes
    '''
    # select only trips to seattle locations
    df = df[seattle_rgcs]
    
    df.fillna(0,inplace=True)
    
    # Add another columns for Seattle City Center, which includes multiple RGCs
    df['City Center'] = df['Seattle Downtown']+df['Seattle First Hill/Capitol Hill']+df['Seattle South Lake Union']\
        +df['Seattle Uptown']
     
    # Save total trips by RGC for later use
    sum_row = df.sum()
    
    # Redefine modes (note: loc[] refers to mode number, iloc[] is an index)

    # SOV: includes "Drive alone"
    df.loc['SOV'] = df.loc[3]    

    # HOV: includes "Drove/rode ONLY with other hh members,"
    # "drove/rode with people not in hh," and vanpool
    df.loc['HOV'] = df.loc[4]+df.loc[5]

    df.loc['Bike'] = df.loc[2]
    df.loc['Walk'] = df.loc[1]
    df.loc['Transit'] = df.loc[6]   
    df = df.loc[['SOV','HOV','Bike','Walk','Transit']]

    # Calculate "other" as the sum of all trips to RGC minus sum of selected modes
    # Other includes all other modes such as taxi, motorcycle, school bus,
    # and also 'other', which falls outside even those categories
    df.loc['Other'] = sum_row - df.sum()
        
    return df

**All trips to/within Seattle RGCs**

In [198]:
all_trips_to_rgc = pd.pivot_table(data=trips, cols='d_rgc_name', index='mode', 
               values='trexpfac', aggfunc='sum')

df = format_pivot(all_trips_to_rgc)
df.to_clipboard()


**All Commute Trips to/within Seattle RGCs**

In [199]:
# Filter for commute trips 

# Since we are concerned with trips to/within RGCs, consider commute trip as home->work trips only
commute_trips = trips[(trips['dpurp'] == 1) & (trips['opurp'] == 0)]

In [201]:
commute_trips_to_rgc = pd.pivot_table(data=commute_trips, cols='d_rgc_name', index='mode', 
               values='trexpfac', aggfunc='sum')

df = format_pivot(commute_trips_to_rgc)
df.to_clipboard()

**Non-commute Trips to/within Seattle RGCs**

In [202]:
# filter for non-work trips ending in RGC zone

# exclude all trips with destination of work
non_commute = trips[-((trips['dpurp'] == 1) & (trips['opurp'] == 0))]

In [203]:
# non-commute trips should be any trip that wasn't defined as a commute trip
len(non_commute)+len(commute_trips)==len(trips)

True

In [204]:
non_commute_to_rgc = pd.pivot_table(data=non_commute, cols='d_rgc_name', index='mode', 
               values='trexpfac', aggfunc='sum')

df = format_pivot(non_commute_to_rgc)
df.to_clipboard()

**All Trips, for Households living in Seattle RGCs**

In [207]:
seattle_rgc_trip_hh = trip_hh[trip_hh['hh_rgc_name'].isin(seattle_rgcs)]

In [208]:
rgc_hh_mode_share = pd.pivot_table(data=seattle_rgc_trip_hh, cols='d_rgc_name', index='mode', 
               values='trexpfac', aggfunc='sum')

df = format_pivot(rgc_hh_mode_share)
df.to_clipboard()

**Non-Work Trips for Households Living in Seattle RGCs**

In [209]:
# This query is for households living in seattle rgcs, but targeted for non-commute trips only

In [210]:
# Since we are no longer considering trips by the destination location,
# we should filter out all commute trips -> this includes home-work and work-home

# First filter out the home->work trips
sea_noncom = seattle_rgc_trip_hh[-((seattle_rgc_trip_hh['dpurp'] == 1) & \
                                   (seattle_rgc_trip_hh['opurp'] == 0))]
print len(sea_noncom)


933


In [211]:
# Next filter those results to remove all work->home trips too!
sea_noncom = sea_noncom[-((sea_noncom['dpurp'] == 0) & \
                                   (sea_noncom['opurp'] == 1))]
print len(sea_noncom)

800


In [212]:
noncommute = pd.pivot_table(data=sea_noncom, cols='d_rgc_name', index='mode', 
               values='trexpfac', aggfunc='sum')

df = format_pivot(noncommute)
df.to_clipboard()

Sample Count:
    to count samples, do the following for each category

In [151]:
# Change aggfunc='sum' to aggfunc='count' in the pivot_table method
all_trips_to_rgc = pd.pivot_table(data=trips, cols='d_rgc_name', index='mode', 
               values='trexpfac', aggfunc='count')

df = format_pivot(all_trips_to_rgc)
df.sum()

d_rgc_name
Seattle Downtown                   2465
Seattle First Hill/Capitol Hill    1295
Seattle Northgate                   460
Seattle South Lake Union            305
Seattle University Community       1005
Seattle Uptown                      386
City Center                        4451
dtype: float64